In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
from itertools import permutations
from sklearn.linear_model import LogisticRegression
from helper import *
import copy
from tqdm import tqdm
import pickle
from sklearn.model_selection import train_test_split

In [227]:
sp22_pre_df = pd.read_csv('./data_by_student/sp22_pre.csv')
sp22_pre_df['exams'] = sp22_pre_df['exams'].apply(literal_eval)

fa22_pre_df = pd.read_csv('./data_by_student/fa22_pre.csv')
fa22_pre_df['exams'] = fa22_pre_df['exams'].apply(literal_eval)

sp23_pre_df = pd.read_csv('./data_by_student/sp23_pre.csv')
sp23_pre_df['exams'] = sp23_pre_df['exams'].apply(literal_eval)

sp22_post_df = pd.read_csv('./data_by_student/sp22_post.csv')
sp22_post_df['exams'] = sp22_post_df['exams'].apply(literal_eval)

fa22_post_df = pd.read_csv('./data_by_student/fa22_post.csv')
fa22_post_df['exams'] = fa22_post_df['exams'].apply(literal_eval)

sp23_post_df = pd.read_csv('./data_by_student/sp23_post.csv')
sp23_post_df['exams'] = sp23_post_df['exams'].apply(literal_eval)



# Check list of courses that can apply the model 
# (i.e., at least one student dropped a class in spring 2022 and fall 2022;
# not all students dropped a class in any semester;
# the course appear in all pre- and post- of sp22, fa22, and sp23)
sp23_pre_exams = list(sp23_pre_df.explode('exams')['exams'].unique())
sp23_post_exams = list(sp23_post_df.explode('exams')['exams'].unique())
sp22_pre_exams = list(sp22_pre_df.explode('exams')['exams'].unique())
fa22_pre_exams = list(fa22_pre_df.explode('exams')['exams'].unique())
sp22_post_exams = list(sp22_post_df.explode('exams')['exams'].unique())
fa22_post_exams = list(fa22_post_df.explode('exams')['exams'].unique())

cur_exams = set(sp23_post_exams).intersection(set(sp23_pre_exams))
# filter exams not in both spring 2022 and fall 2022
past_exams = (set(sp22_pre_exams).intersection(set(sp22_post_exams))).union(set(fa22_pre_exams).intersection(set(fa22_post_exams)))
filtered_exams = set(cur_exams).intersection(past_exams)
# print(filtered_exams)
print(len(filtered_exams))


# Filter out courses with less than 2 dropped students or less than 2 stayed students
filtered_exams2 = copy.copy(filtered_exams)
for course in tqdm(filtered_exams):
    merge = make_feature_data(course)
    num_drop = len(merge[merge['dropped'] == 1])
    if (num_drop <= 1) or (num_drop >= len(merge) - 1) :
        filtered_exams2.remove(course)
        continue
        
    for sem in ['sp22', 'fa22', 'sp23']:
        train_data_df = merge[merge['semester'] == sem]
        if len(train_data_df) == 0:
            break
        num_drop_train = len(train_data_df[train_data_df['dropped'] == 1])
        if (num_drop_train == 0) or (num_drop >= len(train_data_df) -1):
            filtered_exams2.remove(course)
            break
print(len(filtered_exams2))


# Keep only those in engineering
eng_list =['CS', 'INFO', 'ORIE', 'CEE', 'CHEME', 'BME', 'BEE', 'ECE', 'MAE', 'MSE', 
            'AEP', 'ENGRC', 'ENGRD', 'ENGRI', 'ENGRG', 'ENMGT', 'SYSEN', 'STSCI', 
            'MATH', 'PHYS', 'EAS', 'CHEM']
eng_filtered_exams = []
for course in filtered_exams2:
    if course.split('-')[0] in eng_list:
        eng_filtered_exams.append(course)
print('Number of courses remaining: ', len(eng_filtered_exams))
eng_filtered_exams = sorted(eng_filtered_exams)


# Save the list to a file
with open('useful_courses.pkl', 'wb') as f:
    pickle.dump(eng_filtered_exams, f)

478


  0%|▏                                                                                 | 1/478 [00:00<04:22,  1.82it/s]

145 students pre-enrolled in SPAN-1220
7 data point is nan during processing
Sanity check passed
14 students dropped


  0%|▎                                                                                 | 2/478 [00:01<04:33,  1.74it/s]

236 students pre-enrolled in ILRLR-2010
1 data point is nan during processing
Sanity check passed
0 students dropped


...

966 students pre-enrolled in BIOG-1440
1 data point is nan during processing
Sanity check passed
56 students dropped


100%|███████████████████████████████████████████████████████████████████████████████▊| 477/478 [04:27<00:00,  1.69it/s]

436 students pre-enrolled in CS-4670
4 data point is nan during processing
Sanity check passed
44 students dropped


100%|████████████████████████████████████████████████████████████████████████████████| 478/478 [04:27<00:00,  1.78it/s]

475 students pre-enrolled in AEM-2241
9 data point is nan during processing
Sanity check passed
52 students dropped
207
Number of courses remaining:  91


In [5]:
with open('useful_courses.pkl', 'rb') as f:
    eng_filtered_exams = pickle.load(f)